# Documents and Words 
## docclass.py, getwords to extract the features from the text
* Source : https://gist.githubusercontent.com/unlimitedfocus/7200425/raw/c34f7ed8722e40a30caf6efe270504c29ecf7c20/gistfile1.txt

In [ ]:
import re
import math
def getwords(doc):
    splitter=re.compile('\\W*')
    # Split the words by non-alpha characters
    words=[s.lower( ) for s in splitter.split(doc)
        if len(s)>2 and len(s)<20]
    
    # Return the unique set of words only
    return dict([(w,1) for w in words])

## Training the Classifier # Create a class called classifier

In [ ]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()

In [ ]:
cl = classifier(getwords)

In [ ]:
cl.train('the quick brown fox jumps over the lazy dog','good')

In [ ]:
cl.train('make quick money in the online casino','bad')

In [ ]:
cl.fcount('quick','good')

In [ ]:
cl.fcount('quick','bad')

In [ ]:
cl.fcount('casino', 'good')

In [ ]:
cl.fcount('casino', 'bad')

In [ ]:
def sampletrain(cl):
    cl.train('Nobody owns the water.','good')
    cl.train('the quick rabbit jumps fences','good')
    cl.train('buy pharmaceuticals now','bad')
    cl.train('make quick money at the online casino','bad')
    cl.train('the quick brown fox jumps','good')

## Calculating Probabilities

In [ ]:
class classifier2:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    

In [ ]:
cl2 = classifier2(getwords)

In [ ]:
sampletrain(cl2)

In [ ]:
cl2.fprob('quick','good')

### Starting with a Reasonable Guess

In [ ]:
class classifier3:
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)
        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories( )])
        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp
    

In [ ]:
cl3 = classifier3(getwords)

In [ ]:
sampletrain(cl3)

In [ ]:
cl3.weightedprob('money','good',cl3.fprob)

In [ ]:
sampletrain(cl3)

In [ ]:
cl3.weightedprob('money','good',cl3.fprob)

## A Naïve Classifier you need a way to combine the individual word probabilities to get the probability that an entire document belongs in a given category.

Naïve Bayesian classifier: http://en.wikipedia.org/wiki/Naive_Bayes_classifier, Reference: http://blog.sragent.pe.kr/33

### Probability of a Whole Document

In [ ]:
class naivebayes(classifier):
    def __init__(self,getfeatures,filename=None):
        # Counts of feature/category combinations
        self.fc={}
        # Counts of documents in each category
        self.cc={}
        self.getfeatures=getfeatures
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    
    def docprob(self,item,cat):
        features=self.getfeatures(item)
        
        # Multiply the probabilities of all the features together
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
    
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount( )
        docprob=self.docprob(item,cat)
        return docprob*catprob
    
    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)
        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories( )])
        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp
    

### A Quick Introduction to Bayes’ Theorem

In [ ]:
bl = naivebayes(getwords)

In [ ]:
sampletrain(bl)

In [ ]:
bl.prob('quick rabbit','good')

In [ ]:
bl.prob('quick rabbit','bad')

### Choosing a Category

In [ ]:
class naivebayes2(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
        
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
        
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    
    def docprob(self,item,cat):
        features=self.getfeatures(item)
        
        # Multiply the probabilities of all the features together
        p=1
        for f in features: p*=self.weightedprob(f,cat,self.fprob)
        return p
    
    def prob(self,item,cat):
        catprob=self.catcount(cat)/self.totalcount( )
        docprob=self.docprob(item,cat)
        return docprob*catprob
    
    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)
        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories( )])
        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp
    
    def classify(self,item,default=None):
        probs={}
        # Find the category with the highest probability
        
        max=0.0
        for cat in self.categories( ):
            probs[cat]=self.prob(item,cat)            
            if probs[cat]>max:
                max=probs[cat]
            best=cat
        
        # Make sure the probability exceeds threshold*next best
        for cat in probs:
            if cat==best: continue
            if probs[cat]*self.getthreshold(best)>probs[best]: return default
        return best    

In [ ]:
bl2 = naivebayes2(getwords)

In [ ]:
sampletrain(bl2)

In [ ]:
bl2.classify('quick rabbit',default='unknown')

In [ ]:
bl2.classify('quick money',default='unknown')

In [ ]:
bl2.setthreshold('bad',3.0)

In [ ]:
bl2.classify('quick money',default='unknown')

In [ ]:
bl2.classify('quick money',default='unknown')

In [ ]:
for i in range(10):
    sampletrain(bl2)

In [ ]:
bl2.classify('quick money',default='unknown')

### The Fisher Method 

#### alternative method that’s been shown to give very accurate results, particularly for spam filtering.

### Category Probabilities for Features

In [ ]:
class fisherclassifier(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
        
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
        
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    
    def cprob(self,f,cat):
        # The frequency of this feature in this category
        clf=self.fprob(f,cat)
        if clf==0: return 0
        
        # The frequency of this feature in all the categories
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        
        # The probability is the frequency in this category divided by
        # the overall frequency
        p=clf/(freqsum)
        return p
    
    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)
        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories( )])
        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp

In [ ]:
fl = fisherclassifier(getwords)

In [ ]:
sampletrain(fl)

In [ ]:
fl.cprob('quick','good')

In [ ]:
fl.cprob('money','bad')

In [ ]:
fl.weightedprob('money','bad',fl.cprob)

### Combining the Probabilities

In [ ]:
def fisherprob(self,item,cat):
    # Multiply all the probabilities together
    p=1
    features=self.getfeatures(item)
    for f in features:
        p*=(self.weightedprob(f,cat,self.cprob))
        
    # Take the natural log and multiply by -2
    fscore=-2*math.log(p)
    
    # Use the inverse chi2 function to get a probability
    return self.invchi2(fscore,len(features)*2)

In [ ]:
def invchi2(self,chi,df):
    m = chi / 2.0
    sum = term = math.exp(-m)
    for i in range(1, df//2):
        term *= m / i
        sum += term
    return min(sum, 1.0)

In [ ]:
class fisherclassifier2(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.thresholds={}
        
    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
        
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    
    def cprob(self,f,cat):
        # The frequency of this feature in this category
        clf=self.fprob(f,cat)
        if clf==0: return 0
        
        # The frequency of this feature in all the categories
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        
        # The probability is the frequency in this category divided by
        # the overall frequency
        p=clf/(freqsum)
        return p
    
    
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)
        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories( )])
        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp
    
    def fisherprob(self,item,cat):
        # Multiply all the probabilities together
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
            
        # Take the natural log and multiply by -2
        fscore=-2*math.log(p)
        
        # Use the inverse chi2 function to get a probability
        return self.invchi2(fscore,len(features)*2)

    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)
        for i in range(1, df//2):
            term *= m / i
            sum += term
        return min(sum, 1.0)

In [ ]:
fl2 = fisherclassifier2(getwords)

In [ ]:
sampletrain(fl2)

In [ ]:
fl.cprob('quick','good')

In [ ]:
fl2.fisherprob('quick rabbit','good')

In [ ]:
fl2.fisherprob('quick rabbit','bad')

### Classifying Items

In [ ]:
class fisherclassifier3(classifier):
    def __init__(self,getfeatures):
        classifier.__init__(self,getfeatures)
        self.minimums={}
    
    def setminimum(self,cat,min):
        self.minimums[cat]=min
    
    def getminimum(self,cat):
        if cat not in self.minimums: return 0
        return self.minimums[cat]

    def setthreshold(self,cat,t):
        self.thresholds[cat]=t
        
    def getthreshold(self,cat):
        if cat not in self.thresholds: return 1.0
        return self.thresholds[cat]
        
    def train(self,item,cat):
        features=self.getfeatures(item)
        # Increment the count for every feature with this category
        for f in features:
            self.incf(f,cat)
            
        # Increment the count for this category
        self.incc(cat)
        
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
        
    # The number of times a feature has appeared in a category
    def fcount(self,f,cat):
        if f in self.fc and cat in self.fc[f]:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def fprob(self,f,cat):
        if self.catcount(cat)==0: return 0
            # The total number of times this feature appeared in this
            # category divided by the total number of items in this category
        return self.fcount(f,cat)/self.catcount(cat)
    
    def cprob(self,f,cat):
        # The frequency of this feature in this category
        clf=self.fprob(f,cat)
        if clf==0: return 0
        
        # The frequency of this feature in all the categories
        freqsum=sum([self.fprob(f,c) for c in self.categories( )])
        
        # The probability is the frequency in this category divided by
        # the overall frequency
        p=clf/(freqsum)
        return p
        
    def weightedprob(self,f,cat,prf,weight=1.0,ap=0.5):
        # Calculate current probability
        basicprob=prf(f,cat)
        # Count the number of times this feature has appeared in
        # all categories
        totals=sum([self.fcount(f,c) for c in self.categories( )])
        # Calculate the weighted average
        bp=((weight*ap)+(totals*basicprob))/(weight+totals)
        return bp
    
    def fisherprob(self,item,cat):
        # Multiply all the probabilities together
        p=1
        features=self.getfeatures(item)
        for f in features:
            p*=(self.weightedprob(f,cat,self.cprob))
            
        # Take the natural log and multiply by -2
        fscore=-2*math.log(p)
        
        # Use the inverse chi2 function to get a probability
        return self.invchi2(fscore,len(features)*2)

    def invchi2(self,chi,df):
        m = chi / 2.0
        sum = term = math.exp(-m)
        for i in range(1, df//2):
            term *= m / i
            sum += term
        return min(sum, 1.0)

    def classify(self,item,default=None):
        # Loop through looking for the best result
        best=default
        max=0.0
        for c in self.categories( ):
            p=self.fisherprob(item,c)
            
            # Make sure it exceeds its minimum
            if p>self.getminimum(c) and p>max:
                best=c
                max=p
        return best

In [ ]:
fl3 = fisherclassifier3(getwords)

In [ ]:
sampletrain(fl3)

In [ ]:
fl3.classify('quick rabbit')

In [ ]:
fl3.classify('quick money')

In [ ]:
fl3.setminimum('bad',0.8)

In [ ]:
fl3.classify('quick money')

In [ ]:
fl3.setminimum('good',0.4)

In [ ]:
fl3.classify('quick money')

### Persisting the Trained Classifiers, Using SQLite

In [ ]:
from sqlite3 import dbapi2 as sqlite

In [ ]:
def setdb(self,dbfile):
    self.con=sqlite.connect(dbfile)
    self.con.execute('create table if not exists fc(feature,category,count)')
    self.con.execute('create table if not exists cc(category,count)')

In [ ]:
def incf(self,f,cat):
    count=self.fcount(f,cat)
    if count==0:
        self.con.execute("insert into fc values ('%s','%s',1)"% (f,cat))
    else:
        self.con.execute("update fc set count=%d where feature='%s' and category='%s'"% (count+1,f,cat))

def fcount(self,f,cat):
    res=self.con.execute('select count from fc where feature="%s" and category="%s"'%(f,cat)).fetchone( )
    if res==None: return 0
    else: return float(res[0])
    
def incc(self,cat):
    count=self.catcount(cat)
    if count==0:
        self.con.execute("insert into cc values ('%s',1)" % (cat))
    else:
        self.con.execute("update cc set count=%d where category='%s'" % (count+1,cat))
    
def catcount(self,cat):
    res=self.con.execute('select count from cc where category="%s"' %(cat)).fetchone( )
    if res==None: return 0
    else: return float(res[0])
    
def categories(self):
    cur=self.con.execute('select category from cc');
    return [d[0] for d in cur]

def totalcount(self):
    res=self.con.execute('select sum(count) from cc').fetchone( );
    if res==None: return 0
    return res[0]